In [1]:
from functools import partial
from itertools import product

import numpy as np
import pandas as pd

from superfv import EulerSolver
from superfv.initial_conditions import decaying_isotropic_turbulence

In [2]:
def turbulent_crossing_time(sim):
    idx = sim.variable_index_map
    w = sim.snapshots[0]["w"]
    vx = w[idx("vx")]
    vy = w[idx("vy")]
    v = np.sqrt(vx**2 + vy**2)
    crossing_time = 1 / np.std(v)
    return crossing_time

In [3]:
seed = 1
N = 64
T = [0.08, 0.16, 0.24]

base = "../out/isotropic_turbulence_2d_varying_rho_min/"

PAD = {"rho": (0, None)}

configs = {
    "p0": dict(p=0),
    "MUSCL-Hancock": dict(p=1, MUSCL=True, MUSCL_limiter="PP2D"),
    "ZS3": dict(p=3, ZS=True, GL=True, lazy_primitives="adaptive", PAD=PAD),
    "ZS3t": dict(p=3, ZS=True, adaptive_dt=False, lazy_primitives="adaptive", PAD=PAD),
    "MM3": dict(
        p=3,
        MOOD=True,
        lazy_primitives="adaptive",
        limiting_vars=("rho", "vx", "vy"),
        PAD=PAD,
    ),
    "MM3b": dict(
        p=3,
        MOOD=True,
        lazy_primitives="adaptive",
        limiting_vars=("rho", "vx", "vy"),
        PAD=PAD,
        blend=True,
    ),
}

In [ ]:
sims = {}
data = []
for (name, config), M_max in product(configs.items(), [1, 2.5, 5, 10, 25, 50]):
    print(f"- - - - - Running simulation {name} with {M_max=} - - - - -")
    path = f"{base}/{name}/M_max_{M_max}/"

    try:
        with open(path + "status.txt", "r") as f:
            status = f.read().strip()
        data.append(
            dict(
                name=name,
                M_max=M_max,
                passed={"passed": True, "failed": False}.get(status, None),
            )
        )
        print(f"\t Already completed with status: {status}, skipping...")
        continue
    except Exception as e:
        print(f"Error reading status: {e}")
        pass

    sim = EulerSolver(
        ic=partial(decaying_isotropic_turbulence, seed=seed, M=M_max, slope=-5 / 3),
        isothermal=True,
        nx=N,
        ny=N,
        **config,
    )

    try:
        sim.run(
            T,
            q_max=2,
            allow_overshoot=True,
            muscl_hancock=config.get("MUSCL", False),
            log_freq=20,
            path=path,
            overwrite=True,
        )
        sim.print_timings()
        sims[name] = sim
        with open(path + "status.txt", "w") as f:
            f.write("passed")
        data.append(dict(name=name, M_max=M_max, passed=True))
    except Exception as e:
        print(f"Failed: {e}")
        with open(path + "status.txt", "w") as f:
            f.write("failed")
        data.append(dict(name=name, M_max=M_max, passed=False))
        continue
df = pd.DataFrame(data)

- - - - - Running simulation p0 with M_max=1 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation p0 with M_max=2.5 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation p0 with M_max=5 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation p0 with M_max=10 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation p0 with M_max=25 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation p0 with M_max=50 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation MUSCL-Hancock with M_max=1 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation MUSCL-Hancock with M_max=2.5 - - - - -
	 Already completed with status: passed, skipping...
- - - - - Running simulation MUSCL-Hancock with M_max=5 - - - - -
	 Already completed with status: passed, 

In [5]:
df.pivot(index="M_max", columns="name", values="passed").loc[
    :,
    ["MUSCL-Hancock", "ZS3", "ZS3t", "MM3", "MM3b"],
].sort_index(ascending=True)

name,MUSCL-Hancock,ZS3,ZS3t,MM3,MM3b
M_max,,,,,
1.0,True,True,True,True,True
2.5,True,True,True,True,True
5.0,True,True,False,False,True
10.0,True,True,False,False,True
25.0,True,True,False,False,False
50.0,True,True,False,False,True
